In [1]:
#| default_exp core

In [2]:
#| export
import concurrent
import io
import logging
import re
import re2

import cairosvg
import kagglehub
import torch
from lxml import etree
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.models.gemma3 import Gemma3ForConditionalGeneration, Gemma3Processor, Gemma3ForCausalLM

svg_constraints = kagglehub.package_import('metric/svg-constraints')

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Model:
    def __init__(self):
        # Define 4-bit quantization configuration using bitsandbytes for efficient inference
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,                         # Load model in 4-bit precision to save memory
            bnb_4bit_quant_type="nf4",                 # Use the "nf4" quantization type, which improves accuracy over traditional 4-bit formats
            bnb_4bit_use_double_quant=True,            # Enables nested quantization for better memory-efficiency
            bnb_4bit_compute_dtype=torch.bfloat16,     # Use bfloat16 for computation (efficient on modern hardware like A100, RTX 30+)
        )

        # Load the model checkpoint from KaggleHub
        self.model_path = kagglehub.model_download("google/gemma-3/transformers/gemma-3-12b-it")

        # Initialize the processor (tokenizer + preprocessor) for the Gemma 3 model
        self.processor = Gemma3Processor.from_pretrained(self.model_path, use_fast=True)

        # Load the model itself, applying quantization and assigning it automatically to the available device (CPU/GPU)
        self.model = Gemma3ForConditionalGeneration.from_pretrained(self.model_path, 
                                                                    quantization_config=quantization_config,
                                                                    torch_dtype=torch.bfloat16, 
                                                                    device_map="auto",)
        
        # Define the system prompt template for SVG generation
        self.prompt_template = """
Generate a well-formed, minimalist SVG based on the following visual description. The SVG must strictly follow the constraints and produce clean, elegant output optimized for compactness and clarity.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
* **Output Format:** Always use `viewBox="0 0 256 256"` with `width="256"` and `height="256"`
* **Keep output minimal and visually balanced**. Avoid extra elements, comments, or unused attributes.
</constraints>

<example>
<description>"A red circle with a blue square inside"</description>
```svg
<svg viewBox="0 0 256 256" width="256" height="256">
  <circle cx="128" cy="128" r="80" fill="red"/>
  <rect x="88" y="88" width="80" height="80" fill="blue"/>
</svg>
</example>
Respond with only complete, valid SVG code between <svg> and </svg>. Avoid unnecessary detail or repetition. Never output comments or partial code.

<description>"{}"</description>
<svg viewBox="0 0 256 256" width="256" height="256">
"""
        self.default_svg = """<svg width="256" height="256" viewBox="0 0 256 256"><circle cx="50" cy="50" r="40" fill="red" /></svg>"""
        self.constraints = svg_constraints.SVGConstraints()
        self.timeout_seconds = 300

    def predict(self, description: str) -> str:
        def generate_svg():
            try:
                # Format the prompt using the template and user-provided description
                prompt = self.prompt_template.format(description)
                
                # Tokenize the prompt text using the processor (includes tokenizer + any pre-processing)
                # Converts input to tensors and moves to correct device (CPU/GPU)
                inputs = self.processor(text=prompt, return_tensors="pt").to(DEVICE)

                # Disable gradient tracking since we’re only doing inference
                with torch.no_grad():
                    # Generate SVG code using the model with nucleus sampling
                    output = self.model.generate(
                        **inputs,
                        max_new_tokens=1280,      # Limits the maximum number of new tokens to generate
                        do_sample=True,          # Enables random sampling (vs greedy decoding)
                        temperature=0.6,         # Controls randomness (lower = less random)
                        top_p=0.85,               # Enables nucleus sampling (top-p sampling)
                    )

                # Decode the generated tokens to a readable string
                output_decoded = self.processor.decode(output[0],                           # Take the first sequence (batch size = 1)
                                                       skip_special_tokens=True,            # Remove special tokens like <pad>, <bos>, <eos>
                                                       clean_up_tokenization_spaces=False)  # Avoid automatic spacing/cleanup for precision

                
                logging.debug('Output decoded from model: %s', output_decoded)

                matches = re.findall(r"<svg.*?</svg>", output_decoded, re.DOTALL | re.IGNORECASE)
                if matches:
                    svg = matches[-1]
                else:
                    return self.default_svg

                logging.debug('Unprocessed SVG: %s', svg)
                svg = self.enforce_constraints(svg)
                logging.debug('Processed SVG: %s', svg)
                # Ensure the generated code can be converted by cairosvg
                cairosvg.svg2png(bytestring=svg.encode('utf-8'))
                return svg
            except Exception as e:
                logging.error('Exception during SVG generation: %s', e)
                return self.default_svg

        # Execute SVG generation in a new thread to enforce time constraints
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            
            # Submit the generate_svg function to be executed in a separate thread
            future = executor.submit(generate_svg)
            try:

                # Wait for the result with a timeout; if it exceeds, a TimeoutError is raised
                return future.result(timeout=self.timeout_seconds)
                
            except concurrent.futures.TimeoutError:
                
                # If the generation takes longer than the timeout, log a warning and return default SVG
                logging.warning("Prediction timed out after %s seconds.", self.timeout_seconds)
                return self.default_svg
                
            except Exception as e:

                # Catch any other unexpected errors, log them, and return the fallback SVG
                logging.error(f"An unexpected error occurred: {e}")
                return self.default_svg

    def enforce_constraints(self, svg_string: str) -> str:
        """Enforces constraints on an SVG string, removing disallowed elements
        and attributes.

        Parameters
        ----------
        svg_string : str
            The SVG string to process.

        Returns
        -------
        str
            The processed SVG string, or the default SVG if constraints
            cannot be satisfied.
        """
        logging.info('Sanitizing SVG...')

        try:
            parser = etree.XMLParser(remove_blank_text=True, remove_comments=True)
            root = etree.fromstring(svg_string, parser=parser)
        except etree.ParseError as e:
            logging.error('SVG Parse Error: %s. Returning default SVG.', e)
            return self.default_svg
    
        elements_to_remove = []
        for element in root.iter():
            tag_name = etree.QName(element.tag).localname
    
            # Remove disallowed elements
            if tag_name not in self.constraints.allowed_elements:
                elements_to_remove.append(element)
                continue  # Skip attribute checks for removed elements
    
            # Remove disallowed attributes
            attrs_to_remove = []
            for attr in element.attrib:
                attr_name = etree.QName(attr).localname
                if (
                    attr_name
                    not in self.constraints.allowed_elements[tag_name]
                    and attr_name
                    not in self.constraints.allowed_elements['common']
                ):
                    attrs_to_remove.append(attr)
    
            for attr in attrs_to_remove:
                logging.debug(
                    'Attribute "%s" for element "%s" not allowed. Removing.',
                    attr,
                    tag_name,
                )
                del element.attrib[attr]
    
            # Check and remove invalid href attributes
            for attr, value in element.attrib.items():
                 if etree.QName(attr).localname == 'href' and not value.startswith('#'):
                    logging.debug(
                        'Removing invalid href attribute in element "%s".', tag_name
                    )
                    del element.attrib[attr]

            # Validate path elements to help ensure SVG conversion
            if tag_name == 'path':
                d_attribute = element.get('d')
                if not d_attribute:
                    logging.warning('Path element is missing "d" attribute. Removing path.')
                    elements_to_remove.append(element)
                    continue # Skip further checks for this removed element
                # Use regex to validate 'd' attribute format
                path_regex = re2.compile(
                    r'^'  # Start of string
                    r'(?:'  # Non-capturing group for each command + numbers block
                    r'[MmZzLlHhVvCcSsQqTtAa]'  # Valid SVG path commands (adjusted to exclude extra letters)
                    r'\s*'  # Optional whitespace after command
                    r'(?:'  # Non-capturing group for optional numbers
                    r'-?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?'  # First number
                    r'(?:[\s,]+-?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)*'  # Subsequent numbers with mandatory separator(s)
                    r')?'  # Numbers are optional (e.g. for Z command)
                    r'\s*'  # Optional whitespace after numbers/command block
                    r')+'  # One or more command blocks
                    r'\s*'  # Optional trailing whitespace
                    r'$'  # End of string
                )
                if not path_regex.match(d_attribute):
                    logging.warning(
                        'Path element has malformed "d" attribute format. Removing path.'
                    )
                    elements_to_remove.append(element)
                    continue
                logging.debug('Path element "d" attribute validated (regex check).')
        
        # Remove elements marked for removal
        for element in elements_to_remove:
            if element.getparent() is not None:
                element.getparent().remove(element)
                logging.debug('Removed element: %s', element.tag)

        try:
            cleaned_svg_string = etree.tostring(root, encoding='unicode')
            return cleaned_svg_string
        except ValueError as e:
            logging.error(
                'SVG could not be sanitized to meet constraints: %s', e
            )
            return self.default_svg

The following code tests the above model in a local mock-up of this competition's evaluation pipeline. It runs the model on a sample of 15 instances defined in the `test.csv` file in the `kaggle_evaluation` package folder.

In [3]:
import kaggle_evaluation

#logging.basicConfig(level=logging.WARNING, force=True)
#kaggle_evaluation.test(Model)

In [4]:
import polars as pl
import pandas as pd
train = pl.read_csv('/kaggle/input/drawing-with-llms/train.csv')

def generate():

    from IPython.display import SVG
    import time  # Import the time module
    
    logging.basicConfig(level=logging.DEBUG, force=True)
    
    display(train.head())
    
    model = Model()
    svgs = []
    for desc in train.get_column('description'):
        start_time = time.time()  # Record start time
        svg = model.predict(desc)
        end_time = time.time()    # Record end time
        elapsed_time = end_time - start_time # Calculate elapsed time
        
        print(f"Prediction time for description '{desc[:20]}...': {elapsed_time:.4f} seconds") # Print time
        print(svg)
        try:
            #display(SVG(svg))
            print(svg)
        except Exception as e:
            print(e)
            continue

# Uncomment and run the line below to see some generated images
generate()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
train

id,description
str,str
"""02d892""","""a purple forest at dusk"""
"""0dcd2e""","""gray wool coat with a faux fur…"
"""1e9ac1""","""a lighthouse overlooking the o…"
"""2b25db""","""burgundy corduroy pants with p…"
"""4e6a54""","""orange corduroy overalls"""


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dp.kaggle.net:443
DEBUG:urllib3.connectionpool:https://dp.kaggle.net:443 "POST /kaggle-jwt-handler/AttachDatasourceUsingJwtRequest HTTP/1.1" 200 None
DEBUG:bitsandbytes.cextension:Loading bitsandbytes native library from: /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda121.so
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/kaggle/working, stdin=None, shell=False, universal_newlines=False)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/kaggle/working, stdin=None, shell=False, universal_newlines=False)
DEBUG:wandb.docker.auth:Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
DEBUG:wandb.docker.auth:No config file found


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:root:Output decoded from model: 
Generate a well-formed, minimalist SVG based on the following visual description. The SVG must strictly follow the constraints and produce clean, elegant output optimized for compactness and clarity.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
* **Output Format:** Always use `viewBox="0 0 256 256"` with `width="256"` and `height="256"`
* **Keep output minimal and visually balanced**. Avoid extra elements, comments, or unused attributes.
</constraints>

<example>
<description>"A red circle with a blue square inside"</description>
```svg
<svg viewBox="0 0 256 256" width="256" height="256">
  <circle cx="128" cy="128" r="80" fill="red"/>
 

Prediction time for description 'a purple forest at d...': 28.7245 seconds
<svg viewBox="0 0 256 256" width="256" height="256"><path d="M0,256 L256,256 L256,0 L0,0 Z" fill="darkblue"/><polyline points="64,128 128,256 256,64" fill="none" stroke="purple" stroke-width="8"/><polyline points="64,128 128,0 256,192" fill="none" stroke="purple" stroke-width="8"/></svg>
<svg viewBox="0 0 256 256" width="256" height="256"><path d="M0,256 L256,256 L256,0 L0,0 Z" fill="darkblue"/><polyline points="64,128 128,256 256,64" fill="none" stroke="purple" stroke-width="8"/><polyline points="64,128 128,0 256,192" fill="none" stroke="purple" stroke-width="8"/></svg>


...

DEBUG:root:Output decoded from model: 
Generate a well-formed, minimalist SVG based on the following visual description. The SVG must strictly follow the constraints and produce clean, elegant output optimized for compactness and clarity.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
* **Output Format:** Always use `viewBox="0 0 256 256"` with `width="256"` and `height="256"`
* **Keep output minimal and visually balanced**. Avoid extra elements, comments, or unused attributes.
</constraints>

<example>
<description>"A red circle with a blue square inside"</description>
```svg
<svg viewBox="0 0 256 256" width="256" height="256">
  <circle cx="128" cy="128" r="80" fill="red"/>
 

Prediction time for description 'khaki triangles and ...': 18.7218 seconds
<svg viewBox="0 0 256 256" width="256" height="256"><path d="M128 256 L256 128 L0 128 Z" fill="khaki"/><path d="M128 0 L256 128 L0 128 Z" fill="azure"/></svg>
<svg viewBox="0 0 256 256" width="256" height="256"><path d="M128 256 L256 128 L0 128 Z" fill="khaki"/><path d="M128 0 L256 128 L0 128 Z" fill="azure"/></svg>


DEBUG:root:Output decoded from model: 
Generate a well-formed, minimalist SVG based on the following visual description. The SVG must strictly follow the constraints and produce clean, elegant output optimized for compactness and clarity.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
* **Output Format:** Always use `viewBox="0 0 256 256"` with `width="256"` and `height="256"`
* **Keep output minimal and visually balanced**. Avoid extra elements, comments, or unused attributes.
</constraints>

<example>
<description>"A red circle with a blue square inside"</description>
```svg
<svg viewBox="0 0 256 256" width="256" height="256">
  <circle cx="128" cy="128" r="80" fill="red"/>
 

Prediction time for description 'a maroon dodecahedro...': 104.9613 seconds
<svg viewBox="0 0 256 256" width="256" height="256"><polygon points="128 20 165.29 70.91 191.37 125.91 197.37 168.63 165.29 214.71 128 200 90.71 214.71 58.63 168.63 34.71 125.91 20.91 70.91" fill="maroon"/><path d="M128 20L165.29 70.91" stroke="teal" stroke-width="8"/><path d="M165.29 70.91L191.37 125.91" stroke="teal" stroke-width="8"/><path d="M191.37 125.91L197.37 168.63" stroke="teal" stroke-width="8"/><path d="M197.37 168.63L165.29 214.71" stroke="teal" stroke-width="8"/><path d="M165.29 214.71L128 200" stroke="teal" stroke-width="8"/><path d="M128 200L90.71 214.71" stroke="teal" stroke-width="8"/><path d="M90.71 214.71L58.63 168.63" stroke="teal" stroke-width="8"/><path d="M58.63 168.63L34.71 125.91" stroke="teal" stroke-width="8"/><path d="M34.71 125.91L20.91 70.91" stroke="teal" stroke-width="8"/><path d="M20.91 70.91L128 20" stroke="teal" stroke-width="8"/></svg>
<svg viewBox="0 0 256 256" width="256" 

id,description
str,str
"""02d892""","""a purple forest at dusk"""
"""0dcd2e""","""gray wool coat with a faux fur…"
"""1e9ac1""","""a lighthouse overlooking the o…"
"""2b25db""","""burgundy corduroy pants with p…"
"""4e6a54""","""orange corduroy overalls"""
…,…
"""9b71cc""","""a snowy plain"""
"""a395a3""","""black and white checkered pant…"
"""ad4c5c""","""a starlit night over snow-cove…"
